# Initialization

In [ ]:
%load_ext autoreload
%autoreload 2
import sys, warnings, time, numpy, yaml, pandas, logging, os, random
sys.path.append("../src/") # go to parent dir
from data_access import get_X, get_y, get_train_test
from models.factory import ModelFactory
warnings.filterwarnings('ignore')
with open('../confs/logs.yaml', 'rt') as f:
    config = yaml.safe_load(f.read())
logging.config.dictConfig(config)

In [ ]:
X_train, y_train, X_test, y_test, target = get_train_test(train_size=0.8, random_state=42)
train_scores = get_y()
X_submission = get_X('test')
# features=['TEAM_GAME_WON_season_average', 'TEAM_GAME_LOST_season_average', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_CORNERS_5_last_match_average', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_GAME_DRAW_season_average']
features=['TEAM_GAME_WON_season_average', 'TEAM_GAME_LOST_season_average', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_CORNERS_5_last_match_average', 'TEAM_BALL_SAFE_5_last_match_sum']
# features=['TEAM_GAME_WON_season_average', 'TEAM_SAVES_season_average', 'TEAM_GAME_DRAW_season_average', 'TEAM_GOALS_5_last_match_std', 'TEAM_SUBSTITUTIONS_season_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_PENALTIES_5_last_match_average', 'TEAM_SAVES_5_last_match_average', 'TEAM_SHOTS_TOTAL_5_last_match_std']

features=['TEAM_GAME_WON_season_average', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_SAVES_5_last_match_sum', 'TEAM_YELLOWCARDS_5_last_match_std', 'TEAM_GAME_DRAW_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_FOULS_5_last_match_sum', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SAVES_season_sum', 'TEAM_CORNERS_season_std', 'TEAM_PASSES_5_last_match_std', 'TEAM_ATTACKS_season_sum', 'TEAM_FOULS_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_sum', 'TEAM_SUBSTITUTIONS_season_average', 'TEAM_CORNERS_5_last_match_sum']
features=['TEAM_GAME_WON_season_average', 'TEAM_SAVES_season_average', 'TEAM_GAME_DRAW_season_average', 'TEAM_GOALS_5_last_match_std', 'TEAM_SUBSTITUTIONS_season_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_PENALTIES_5_last_match_average', 'TEAM_SAVES_5_last_match_average', 'TEAM_SHOTS_TOTAL_5_last_match_std']
features=['TEAM_DANGEROUS_ATTACKS_5_last_match_average', 'TEAM_FOULS_season_average', 'TEAM_REDCARDS_season_sum', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_SUBSTITUTIONS_season_std', 'TEAM_ATTACKS_5_last_match_average', 'TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_TOTAL_5_last_match_std']
features=['TEAM_SAVES_5_last_match_average', 'TEAM_BALL_SAFE_season_std', 'TEAM_SAVES_5_last_match_std', 'TEAM_BALL_SAFE_season_average', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_season_sum', 'TEAM_GAME_WON_season_average', 'TEAM_GOALS_season_average', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_SAVES_season_std', 'TEAM_GAME_LOST_season_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_OFFSIDES_5_last_match_std', 'TEAM_SHOTS_ON_TARGET_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_season_sum', 'TEAM_SAVES_season_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_GAME_LOST_season_sum', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_GOALS_season_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_average', 'TEAM_SAVES_season_sum', 'TEAM_GOALS_5_last_match_std', 'TEAM_SAVES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_SUBSTITUTIONS_5_last_match_average']
features=['TEAM_GAME_WON_season_sum', 'TEAM_GAME_LOST_season_sum', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_SHOTS_ON_TARGET_season_average']
X_train = X_train[features]
X_test = X_test[features]
X_submission = X_submission[features]

# Methods

In [ ]:
def eval_model(model, x_train, x_test, x_submission, save_model=False, save_proba=False):
    start = time.time()
    model.fit()
    score = model.evaluate(x_test)
    if save_proba:
        def save_model(model, x, root_path):
            y = model.predict(x)
            y.columns = ['HOME_WINS', 'DRAW', 'AWAY_WINS']
            y = model.format_y(y)
            y.to_csv(f'{root_path}{model.name}.csv', index=False)
        save_model(model, x_train, '../data/output/mix/train/')
        save_model(model, x_test, '../data/output/mix/test/')
        save_model(model, x_submission, '../data/output/mix/submission/')
    end = time.time()
    logging.info(f'{model.name}={score} in {numpy.round((end-start), 2)}s')
    if save_model:
        model.save(x_submission)
    return {'name': model.name, 'score': score, 'time': numpy.round((end-start), 2)}

In [ ]:
%load_ext autoreload
%autoreload 2
def eval_model_for_name(name, x_train, x_test, x_submission, save_proba=False, save_model=False):
    with open('../confs/models.yaml', 'r') as file:
        configurations = yaml.safe_load(file)
    factory = ModelFactory(configurations, x_train, y_train, train_scores)
    model = factory.get_model(name)
    return eval_model(model, x_train=x_train, x_test=x_test, x_submission=x_submission, save_proba=save_proba, save_model=save_model)

In [ ]:
def get_features(name = 'random_forest'):
    with open('../confs/models.yaml', 'r') as file:
        configurations = yaml.safe_load(file)
    factory = ModelFactory(configurations, X_train, y_train, train_scores)
    model = factory.get_model(name)
    eval_model(model)
    df_importances = model.get_feature_importances()
    return list(df_importances['feature'])

# Evaluation

In [ ]:
%load_ext autoreload
%autoreload 2
names = ['dummy', 'random_forest']
# names= ['xgb_gblinear', 'catboost', 'random_forest', 'gradient_boosting', 'ada_boost', 'extra_trees', 'hist_gradient_boosting']
names= ['xgb_classifier']
# names= ['lightgbm', 'catboost', 'random_forest', 'gradient_boosting', 'ada_boost', 'extra_trees', 'decision_tree', 'hist_gradient_boosting', 'mlp', 'kneighbors', 'xgb_classifier']
for name in names:
    eval_model_for_name(name, x_train=X_train, x_test=X_test, x_submission=X_submission, save_proba=True, save_model=False)

In [ ]:
with open('../confs/models.yaml', 'r') as file:
    configurations = yaml.safe_load(file)
name='ada_boost'
size=len(features)-2
factory = ModelFactory(configurations, X_train, y_train, train_scores)
model = factory.get_model(name)
model.fit()
score = model.evaluate(X_test)
print(f'score={score}')
for i in range(0, 50):
    random_features=random.sample(features, size)
    factory = ModelFactory(configurations, X_train[random_features], y_train, train_scores)
    model = factory.get_model(name)
    model.fit()
    score = model.evaluate(X_test[random_features])
    logging.info(f'score={score}, i={i}, random_features={random_features}')

In [ ]:
%load_ext autoreload
%autoreload 2
save_model = True
with open('../confs/models.yaml', 'r') as file:
    configurations = yaml.safe_load(file)
factory = ModelFactory(configurations, X_train, y_train, train_scores)
lines = []
for model in factory.get_models():
    model.fit()
    score = model.evaluate(X_test)
    lines.append(eval_model(model, x_train=X_train, x_test=X_test, x_submission=X_submission, save_model=save_model))
df = pandas.DataFrame(lines)
df = df.sort_values(by=['score'], ascending=False)
df=df.sort_values(by=['score'], ascending=False)
df.to_csv('../data/output/result.csv')

In [ ]:
print(df.sort_values(by=['score'], ascending=False))
df.to_csv('../data/output/result.csv')

# Let's boogy !

In [ ]:
def get_mix_X(X, folder = '../data/input/train/'):
    df = None
    df = X.copy()
    for filename in os.listdir(folder):
        model = filename.split('.')[0]
        df_tmp = pandas.read_csv(f'{folder}{filename}')
        df_tmp.columns = [model]
        if df is None:
            df = df_tmp
        else:
            df[df_tmp.columns] = df_tmp
    return df

In [ ]:
%load_ext autoreload
%autoreload 2
names = ['dummy', 'random_forest']
# names= ['xgb_gblinear', 'catboost', 'random_forest', 'gradient_boosting', 'ada_boost', 'extra_trees', 'hist_gradient_boosting']
names= ['random_forest', 'catboost', 'gradient_boosting', 'ada_boost', 'lightgbm', 'extra_trees', 'decision_tree', 'hist_gradient_boosting', 'mlp', 'kneighbors', 
        'xgb_gblinear', 'xgb_classifier', 'keras_relu_1', 'xgb_gbtree']
names= ['random_forest', 'catboost', 'gradient_boosting', 'ada_boost', 'lightgbm', 'extra_trees', 'decision_tree', 'hist_gradient_boosting', 'mlp', 'kneighbors', 
        'xgb_gblinear', 'xgb_classifier', 'xgb_gbtree']
# names= ['xgb_gblinear']
x_train = get_mix_X(X_train, '../data/output/mix/train/')
x_test = get_mix_X(X_test, '../data/output/mix/test/')
x_submission = get_mix_X(X_submission, '../data/output/mix/submission/')

with open('../confs/models.yaml', 'r') as file:
    configurations = yaml.safe_load(file)
factory = ModelFactory(configurations, x_train, y_train, train_scores)
lines = []
for name in names:
    lines.append(eval_model_for_name(name, x_train=x_train, x_test=x_test, x_submission=x_submission, save_model=True))
df = pandas.DataFrame(lines)
df = df.sort_values(by=['score'], ascending=False)
df.to_csv('../data/output/boogy_result.csv')

In [ ]:
print(df.sort_values(by=['score'], ascending=False))